In [1]:
import pandas as pd
import spacy
import steamreviews
import json
import csv
import os
import glob
import json
from pandas.io.json import json_normalize

## Reviews download
https://pypi.org/project/steamreviews/

In [2]:
list_of_app_ids = [105600, 413150, 219150, 261570, 1091500, 381210, 367520, 304430]
def get_reviews(list_of_app_ids):
    request_params = dict()
    request_params['language'] = 'english'
    request_params['day_range'] = '60'
    request_params['filter'] = 'all'
    for app_id in list_of_app_ids:
        review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id,chosen_request_params=request_params)
    return review_dict, query_count
get_reviews(list_of_app_ids)

[appID = 105600] expected #reviews = 575255
[appID = 413150] expected #reviews = 295520
[appID = 219150] expected #reviews = 37878
[appID = 261570] expected #reviews = 14248
[appID = 1091500] expected #reviews = 253070
[appID = 381210] expected #reviews = 254420
[appID = 367520] expected #reviews = 109431
[appID = 304430] expected #reviews = 19060


({'reviews': {'134365818': {'recommendationid': '134365818',
    'author': {'steamid': '76561198085497274',
     'num_games_owned': 843,
     'num_reviews': 512,
     'playtime_forever': 100,
     'playtime_last_two_weeks': 0,
     'playtime_at_review': 100,
     'last_played': 1663267739},
    'language': 'english',
    'review': 'The game follows a young boy who must navigate through a dark and dystopian world filled with danger and mystery. The gameplay consists of solving puzzles and overcoming obstacles by manipulating the environment and interacting with various objects and creatures.\n\nOne of the standout features of "Inside" is its stunning visual design. The game\'s art style is simplistic yet incredibly effective, using a dark and moody color palette to create a sense of unease and tension. The attention to detail in the environment is also impressive, with every area of the game feeling meticulously crafted and full of hidden secrets.\n\nThe game\'s sound design is equally 

In [3]:
dir_name = "data"
reviews_jsons = glob.glob(f"{dir_name}/*")
reviews_jsons

['data\\review_105600.json',
 'data\\review_1091500.json',
 'data\\review_219150.json',
 'data\\review_261570.json',
 'data\\review_304430.json',
 'data\\review_329070.json',
 'data\\review_367520.json',
 'data\\review_381210.json',
 'data\\review_413150.json']

In [4]:
title_dict = {"105600":"Terraria",
              "1091500": "Cyberpunk 2077",
              "219150": "Hotline Miami",
              "261570":"Ori and the Blind Forest",
              "381210":"Dead by Daylight",
              "413150":"Stardew Valley",
              "367520":"Hollow Knight",
              "304430": "INSIDE"
             }

In [5]:
data_list = []
for i in reviews_jsons:
    with open(i, encoding = "utf-8") as json_file:
        json_data = json.load(json_file)
        reviews = json_data['reviews']
        file_name = json_file.name.split('.')[0].split('_')[1]
        for key in reviews.keys():
            data_list.append([title_dict.get(file_name),reviews[key]['author']['steamid'], reviews[key]['votes_up'], 
                          reviews[key]['review'], reviews[key]['weighted_vote_score'],  
                          reviews[key]['author']['playtime_at_review'], reviews[key]['author']['num_games_owned'],
                          reviews[key]['author']['num_reviews'], reviews[key]["timestamp_updated"], 
                          reviews[key]["timestamp_created"] ])
df = pd.DataFrame(data_list, columns = ["title","steamid", "votes_up", "review", "weighted_vote_score", 
                                           "playtime_at_review", "num_games_owned", "num_reviews", "timestamp_updated",
                                           "timestamp_created"
                                          ])
df["title"].value_counts()

Dead by Daylight            747
Cyberpunk 2077              468
Hollow Knight               404
Terraria                    371
Stardew Valley              331
INSIDE                      200
Hotline Miami               178
Ori and the Blind Forest    122
Name: title, dtype: int64

In [11]:
df.sample(frac=0.2).head()

,title,steamid,votes_up,review,weighted_vote_score,playtime_at_review,num_games_owned,num_reviews,timestamp_updated,timestamp_created
30,Terraria,76561198798595534,1,game got corrupted rip 150 hour character and ...,0.523809552192687988,9253,15,2,1682483878,1682483878
306,Terraria,76561198875372235,13,Big Slime died me :(,0.608997523784637451,1206,68,40,1678780354,1678780354
712,Cyberpunk 2077,76561198146425708,6891,"McDonald's Ads - The game\n\nYou see a juicy, ...",0.898983120918273926,2125,221,7,1613483017,1612858264
1623,Hollow Knight,76561198290740148,1,i love how complex the game mechanics and figh...,0.523809552192687988,694,21,8,1679757286,1679757286
683,Cyberpunk 2077,76561197996200954,2059,"I know what you're thinking. Like me, you prob...",0.928052842617034912,2991,606,3,1643424406,1643424406


## Preprocessing